In [1]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import pandas as pd
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Activation, Dropout, Flatten, Dense, merge, Reshape, Permute, Multiply, Dot,dot, Concatenate, Add
from keras.layers import Input
from keras import backend as K
from keras.engine.topology import Layer
import keras as keras

# packages for learning from crowds
from crowd_layer.crowd_layers import CrowdsClassification, MaskedMultiCrossEntropy, CrowdsClassificationSModel, \
    CrowdsClassificationCModelSingleWeight, CrowdsClassificationCModel, MaskedMultiCrossEntropyCosSim, \
    MaskedMultiCrossEntropyBaseChannel, MaskedMultiCrossEntropyBaseChannelConst, CrowdsClassificationSModelChannelMatrix, \
    MaskedMultiCrossEntropyCurriculumChannelMatrix
from crowd_layer.crowd_aggregators import CrowdsCategoricalAggregator

/home/yajingyang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# prevent tensorflow from allocating the entire GPU memory at once
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

In [3]:
# Prepare data
def load_data(filename):
    f = open(filename, 'rb')
    data = np.load(f)
    f.close()
    return data

def one_hot(target, n_classes):
    targets = np.array([target]).reshape(-1)
    one_hot_targets = np.eye(n_classes)[targets]
    return one_hot_targets

def get_data(DATA_PATH, N_CLASSES):
    
    print("\nLoading train data...")
    # images processed by VGG16
    data_train_vgg16 = load_data(DATA_PATH+"data_train.npy")
    print(data_train_vgg16.shape)

    # ground truth labels
    labels_train = load_data(DATA_PATH+"labels_train.npy")
    print(labels_train.shape)

    # labels obtained from majority voting
    labels_train_mv = load_data(DATA_PATH+"labels_train_mv.npy")
    print(labels_train_mv.shape)

#     # labels obtained by using the approach by Dawid and Skene
#     labels_train_ds = load_data(DATA_PATH+"labels_train_DS.npy")
#     print(labels_train_ds.shape)

    # data from Amazon Mechanical Turk
    print("\nLoading AMT data...")
    answers = load_data(DATA_PATH+"answers.npy")
    print(answers.shape)
    N_ANNOT = answers.shape[1]
    print("N_CLASSES:", N_CLASSES)
    print("N_ANNOT:", N_ANNOT)

    # load test data
    print("\nLoading test data...")

    # images processed by VGG16
    data_test_vgg16 = load_data(DATA_PATH+"data_test.npy")
    print(data_test_vgg16.shape)

    # test labels
    labels_test = load_data(DATA_PATH+"labels_test.npy")
    print(labels_test.shape)

    print("\nLoading validation data...")
    # images processed by VGG16
    data_valid_vgg16 = load_data(DATA_PATH+"data_valid.npy")
    print(data_valid_vgg16.shape)

    # validation labels
    labels_valid = load_data(DATA_PATH+"labels_valid.npy")
    print(labels_valid.shape)

    labels_train_bin = one_hot(labels_train, N_CLASSES)
    labels_train_mv_bin = one_hot(labels_train_mv, N_CLASSES)
#     labels_train_ds_bin = one_hot(labels_train_ds, N_CLASSES)
#     print(labels_train_ds_bin.shape)
    labels_test_bin = one_hot(labels_test, N_CLASSES)
    labels_valid_bin = one_hot(labels_valid, N_CLASSES)


    answers_bin_missings = []
    for i in range(len(answers)):
        row = []
        for r in range(N_ANNOT):
            if answers[i,r] == -1:
                row.append(-1 * np.ones(N_CLASSES))
            else:
                row.append(one_hot(answers[i,r], N_CLASSES)[0,:])
        answers_bin_missings.append(row)
    answers_bin_missings = np.array(answers_bin_missings).swapaxes(1,2)

    answers_test_bin_missings = np.zeros((len(labels_test), N_CLASSES))
    answers_test_bin_missings[np.arange(len(labels_test)), labels_test] = 1
    answers_test_bin_missings = np.repeat(answers_test_bin_missings.reshape([len(labels_test),N_CLASSES,1]), N_ANNOT, axis=2)

    answers_valid_bin_missings = np.zeros((len(labels_valid), N_CLASSES))
    answers_valid_bin_missings[np.arange(len(labels_valid)), labels_valid] = 1
    answers_valid_bin_missings = np.repeat(answers_valid_bin_missings.reshape([len(labels_valid),N_CLASSES,1]), N_ANNOT, axis=2)
    
    x = {'train': data_train_vgg16, 'test': data_test_vgg16, 'val': data_valid_vgg16}
    y_gt = {'train': labels_train_bin, 'test': labels_test_bin, 'val': labels_valid_bin}
    y_annot = {'train': answers_bin_missings, 'test': answers_test_bin_missings, 'val': answers_valid_bin_missings, 'mv':labels_train_mv_bin}
    return x, y_gt, y_annot

In [4]:
# Build model
def eval(model,x_test, y_test):
    print('Test dataset results: ')
    print(dict(zip(model.metrics_names,model.evaluate(x_test,y_test, verbose=False))))


def get_trace(model):

    channel_matrix = model.get_weights()[-1]
    channel_matrix_trace = tf.trace(K.permute_dimensions(channel_matrix, [2, 0, 1]))
    channel_matrix_trace_arr = K.eval(channel_matrix_trace)
    return channel_matrix_trace_arr


def print_single_loss(model):
    import matplotlib.pyplot as plt

    # list all data in history
    print(model.history.keys())
    # summarize history for accuracy
    plt.plot(model.history['baseline_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.show()
    # summarize history for loss
    plt.plot(model.history['baseline_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.show()


def print_history(df, title):
    import matplotlib.pyplot as plt
    # Make a data frame
    df['x'] = range(df.shape[0])

    # style
    plt.style.use('seaborn-darkgrid')

    # create a color palette
    palette = plt.get_cmap('Set1')

    # multiple line plot
    num = 0
    for column in df.drop('x', axis=1):
        num += 1
        plt.plot(df['x'], df[column], marker='', color=palette(num), linewidth=1, alpha=0.9, label=column)

    # Add legend
    plt.legend(loc=2, ncol=2)

    # Add titles
    plt.title(title, loc='left', fontsize=12, fontweight=0, color='orange')
    plt.xlabel("Time")
    plt.ylabel("Score")
    plt.savefig(title+'.png')

In [5]:
def build_base_model(train_data_shape, N_CLASSES):
    base_model = Sequential()
    base_model.add(Conv2D(32, (3, 3), input_shape=train_data_shape[1:]))
    base_model.add(Activation('relu'))
    base_model.add(MaxPooling2D(pool_size=(2, 2)))
    base_model.add(BatchNormalization(scale=False))

    base_model.add(Conv2D(32, (3, 3)))
    base_model.add(Activation('relu'))
    base_model.add(MaxPooling2D(pool_size=(2, 2)))

    base_model.add(Conv2D(64, (3, 3)))
    base_model.add(Activation('relu'))
    base_model.add(MaxPooling2D(pool_size=(2, 2)))

    base_model.add(Conv2D(64, (3, 3)))
    base_model.add(Activation('relu'))
    base_model.add(MaxPooling2D(pool_size=(2, 2)))

    base_model.add(Flatten()) 
    base_model.add(Dense(128))
    base_model.add(Activation('relu'))
    base_model.add(Dropout(0.5))
    base_model.add(Dense(N_CLASSES))
    base_model.add(Activation('softmax'))
    base_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return base_model

In [6]:
def majority_vote(x, y_gt, y_annot, N_CLASSES):
    train_data_shape = x['train'].shape
    N_ANNOT = y_annot['train'].shape[2]
    baseline_model = build_base_model(train_data_shape, N_CLASSES)

#     eval(baseline_model, x['test'], y_test=y_gt['test'])
    history = baseline_model.fit(x['train'], y_annot['mv'], epochs=N_EPOCHS, shuffle=True,
                              batch_size=BATCH_SIZE, verbose=0)
    eval(baseline_model, x['test'], y_test=y_gt['test'])
    return history

In [7]:
def baseline_gt(x, y_gt, N_CLASSES):
    train_data_shape = x['train'].shape
    baseline_model = build_base_model(train_data_shape, N_CLASSES)

#     eval(baseline_model, x['test'], y_test=y_gt['test'])
    history = baseline_model.fit(x['train'], y_gt['train'], epochs=N_EPOCHS, shuffle=True,
                                 batch_size=BATCH_SIZE, verbose=0)
    eval(baseline_model, x['test'], y_test=y_gt['test'])
    return history

In [8]:
def build_base_crowd_model(train_data_shape, N_CLASSES, N_ANNOT, softmax, trace):
    base_model = Sequential()
    base_model.add(Conv2D(32, (3, 3), input_shape=train_data_shape[1:]))
    base_model.add(Activation('relu'))
    base_model.add(MaxPooling2D(pool_size=(2, 2)))
    base_model.add(BatchNormalization(scale=False))

    base_model.add(Conv2D(32, (3, 3)))
    base_model.add(Activation('relu'))
    base_model.add(MaxPooling2D(pool_size=(2, 2)))

    base_model.add(Conv2D(64, (3, 3)))
    base_model.add(Activation('relu'))
    base_model.add(MaxPooling2D(pool_size=(2, 2)))

    base_model.add(Conv2D(64, (3, 3)))
    base_model.add(Activation('relu'))
    base_model.add(MaxPooling2D(pool_size=(2, 2)))

    base_model.add(Flatten()) 
    base_model.add(Dense(128))
    base_model.add(Activation('relu'))
    base_model.add(Dropout(0.5))
    
    train_inputs = Input(shape=(train_data_shape[1:]))
    last_hidden = base_model(train_inputs)
    baseline_output = Dense(N_CLASSES, activation='softmax', name='baseline')(last_hidden)

    if softmax:
        channel_layer = CrowdsClassificationSModelChannelMatrix(N_CLASSES, N_ANNOT)
        channeled_output = channel_layer([last_hidden, baseline_output])
    else:
        channel_layer = CrowdsClassification(N_CLASSES, N_ANNOT)
        channeled_output = channel_layer(baseline_output)

    model = Model(inputs=train_inputs, outputs=[channeled_output, baseline_output])

    if trace:
        loss = MaskedMultiCrossEntropyCurriculumChannelMatrix(model, 1, 1).loss
    else:
        loss = MaskedMultiCrossEntropy().loss

    # compile model with masked loss and train
    model.compile(optimizer='adam',
                         loss=[loss, 'categorical_crossentropy'],
                         loss_weights=[1, 0],
                         metrics=['accuracy']
                        )
    return model

In [9]:
def crowd_model(x, y_gt, y_annot, N_CLASSES, softmax, trace):
    train_data_shape = x['train'].shape
    N_ANNOT = y_annot['train'].shape[2]

    model = build_base_crowd_model(train_data_shape, N_CLASSES, N_ANNOT, softmax, trace)    
    
    history = model.fit(x['train'], [y_annot['train'], y_gt['train']], epochs=N_EPOCHS, shuffle=True,
                              batch_size=BATCH_SIZE, verbose=0)
    trace_arr = get_trace(model)
    eval(model, x['test'], y_test=[y_annot['test'], y_gt['test']])
    return history, trace_arr

In [14]:
def crowd_model_pretrain_with_clean_data(x, y_gt, y_annot, x_sample, y_gt_sample, N_CLASSES, softmax, trace):
    train_data_shape = x['train'].shape
    N_ANNOT = y_annot['train'].shape[2]

    base_model = Sequential()
    base_model.add(Conv2D(32, (3, 3), input_shape=train_data_shape[1:]))
    base_model.add(Activation('relu'))
    base_model.add(MaxPooling2D(pool_size=(2, 2)))
    base_model.add(BatchNormalization(scale=False))

    base_model.add(Conv2D(32, (3, 3)))
    base_model.add(Activation('relu'))
    base_model.add(MaxPooling2D(pool_size=(2, 2)))

    base_model.add(Conv2D(64, (3, 3)))
    base_model.add(Activation('relu'))
    base_model.add(MaxPooling2D(pool_size=(2, 2)))

    base_model.add(Conv2D(64, (3, 3)))
    base_model.add(Activation('relu'))
    base_model.add(MaxPooling2D(pool_size=(2, 2)))

    base_model.add(Flatten()) 
    base_model.add(Dense(128))
    base_model.add(Activation('relu'))
    base_model.add(Dropout(0.5))

    train_inputs = Input(shape=(train_data_shape[1:]))
    last_hidden = base_model(train_inputs)
    baseline_output = Dense(N_CLASSES, activation='softmax', name='baseline')(last_hidden)

    if softmax:
        channel_layer = CrowdsClassificationSModelChannelMatrix(N_CLASSES, N_ANNOT)
        channeled_output = channel_layer([last_hidden, baseline_output])
    else:
        channel_layer = CrowdsClassification(N_CLASSES, N_ANNOT)
        channeled_output = channel_layer(baseline_output)

    baseline_model = Model(inputs=train_inputs, outputs=baseline_output)

    # compile model with masked loss and train
    baseline_model.compile(optimizer='adam',
                         loss='categorical_crossentropy',
                         metrics=['accuracy']
                        )
    
    history = baseline_model.fit(x_sample['train'], y_gt_sample['train'], epochs=N_EPOCHS, shuffle=True,
                              batch_size=BATCH_SIZE, verbose=0)
    
    model = Model(inputs=train_inputs, outputs=[channeled_output, baseline_output])
    
    if trace:
        loss = MaskedMultiCrossEntropyCurriculumChannelMatrix(model, 1, 1).loss
    else:
        loss = MaskedMultiCrossEntropy().loss
    # compile model with masked loss and train
    model.compile(optimizer='adam',
                         loss=[loss, 'categorical_crossentropy'],
                         loss_weights=[1, 0],
                         metrics=['accuracy']
                        )
    model.set_weights(baseline_model.get_weights()) 

    history = model.fit(x['train'], [y_annot['train'], y_gt['train']], epochs=N_EPOCHS, shuffle=True,
                              batch_size=BATCH_SIZE, verbose=0)
    trace_arr = get_trace(model)
    eval(model, x['test'], y_test=[y_annot['test'], y_gt['test']])
    return history, trace_arr

In [11]:
def get_clean_data(DATA_PATH, N_CLASSES):
    print("\nLoading train data...")
    # images processed by VGG16
    data_train = load_data(DATA_PATH+"data_train.npy")
    print(data_train.shape)

    # ground truth labels
    labels_train = load_data(DATA_PATH+"labels_train.npy")
    print(labels_train.shape)

    # load test data
    print("\nLoading test data...")

    # images processed by VGG16
    data_test = load_data(DATA_PATH+"data_test.npy")
    print(data_test.shape)

    # test labels
    labels_test = load_data(DATA_PATH+"labels_test.npy")
    print(labels_test.shape)

    print("\nLoading validation data...")
    # images processed by VGG16
    data_valid = load_data(DATA_PATH+"data_valid.npy")
    print(data_valid.shape)

    # validation labels
    labels_valid = load_data(DATA_PATH+"labels_valid.npy")
    print(labels_valid.shape)

    labels_train_bin = one_hot(labels_train, N_CLASSES)
#     labels_train_ds_bin = one_hot(labels_train_ds, N_CLASSES)
#     print(labels_train_ds_bin.shape)
    labels_test_bin = one_hot(labels_test, N_CLASSES)
    labels_valid_bin = one_hot(labels_valid, N_CLASSES)
    
    x = {'train': data_train, 'test': data_test, 'val': data_valid}
    y = {'train': labels_train_bin, 'test': labels_test_bin, 'val': labels_valid_bin}
    return x, y

In [16]:
NUM_RUNS = 20
N_CLASSES = 2
BATCH_SIZE = 64
N_EPOCHS = 35
W = 0

DATA_PATH = "/home/yajingyang/Downloads/PetImages/"
x, y_gt, y_annot = get_data(DATA_PATH, N_CLASSES)
N_ANNOT = y_annot['train'].shape[2]

for clean_percent in [0.01, 0.02, 0.03, 0.04, 0.05]:
    x_sample = x.copy()
    x_sample['train'] = x_sample['train'][:round(x_sample['train'].shape[0]*clean_percent)]
    y_gt_sample = y_gt.copy()
    y_gt_sample['train'] = y_gt_sample['train'][:round(y_gt_sample['train'].shape[0]*clean_percent)]
    y_annot_mix = y_annot.copy()
    y_annot_mix['train'][:round(y_annot_mix['train'].shape[0]*clean_percent)] \
        = np.reshape(np.repeat(y_gt['train'][:round(y_annot_mix['train'].shape[0]*clean_percent)], N_ANNOT, axis=1)
                     , (-1, N_CLASSES, N_ANNOT))
    
    print(x_sample['train'].shape, y_gt_sample['train'].shape, y_annot_mix['train'].shape)

    prefix = 'pct_%.2f_'%clean_percent
    loss_csv = prefix + 'loss.csv'
    acc_csv = prefix + 'acc.csv'
    trace_csv = prefix + 'trace.csv'
    acc_df = pd.DataFrame()
    loss_df = pd.DataFrame()
    trace_df = pd.DataFrame()

#     print('\nBaseline model with %.2f clean data' % (clean_percent))
#     for i in range(NUM_RUNS):
#         clean_base_acc_df = pd.DataFrame()
#         clean_base_loss_df = pd.DataFrame()
#         clean_history = baseline_gt(x_sample, y_gt_sample, N_CLASSES)
#         clean_base_acc_df.loc[:, i] = clean_history.history['acc']
#         clean_base_loss_df.loc[:, i] = clean_history.history['loss']
            
#     print('\nCrowd noise adaptation model with %.2f clean data' % (clean_percent))
#     for i in range(NUM_RUNS):
#         acc_df = pd.DataFrame()
#         loss_df = pd.DataFrame()
#         history, trace_arr = crowd_model(x, y_gt, y_annot_mix, N_CLASSES, False, True)
#         acc_df.loc[:, i] = history.history['baseline_acc']
#         loss_df.loc[:, i] = history.history['baseline_loss']
        
    print('\nCrowd noise adaptation model pretrain with %.2f clean data' % (clean_percent))
    for i in range(NUM_RUNS):
        acc_df = pd.DataFrame()
        loss_df = pd.DataFrame()
        history, trace_arr = crowd_model_pretrain_with_clean_data(x, y_gt, y_annot, x_sample, y_gt_sample, N_CLASSES, False, True)
        acc_df.loc[:, i] = history.history['baseline_acc']
        loss_df.loc[:, i] = history.history['baseline_loss']    


Loading train data...
(12499, 150, 150, 3)
(12499,)
(12499,)

Loading AMT data...
(12499, 200)
N_CLASSES: 2
N_ANNOT: 200

Loading test data...
(6250, 150, 150, 3)
(6250,)

Loading validation data...
(6249, 150, 150, 3)
(6249,)
(125, 150, 150, 3) (125, 2) (12499, 2, 200)

Crowd noise adaptation model pretrain with 0.01 clean data
Tensor("baseline_1/Softmax:0", shape=(?, 2), dtype=float32)
<tf.Variable 'crowds_classification_2/CrowdLayer:0' shape=(2, 2, 200) dtype=float32_ref>
Tensor("crowds_classification_2/Reshape_2:0", shape=(?, 2, 200), dtype=float32)
(2,)
Test dataset results: 
{'loss': -1.3057886334609985, 'crowds_classification_2_loss': -1.3057886334609985, 'baseline_loss': 0.6937303191757203, 'crowds_classification_2_acc': 0.0, 'baseline_acc': 0.4969600000190735}
Tensor("baseline_2/Softmax:0", shape=(?, 2), dtype=float32)
<tf.Variable 'crowds_classification_3/CrowdLayer:0' shape=(2, 2, 200) dtype=float32_ref>
Tensor("crowds_classification_3/Reshape_2:0", shape=(?, 2, 200), dtype

KeyboardInterrupt: 